# Experiments with Runnables

Create a few RunnableLambda, and compose them in sequence and in parallel

In [3]:

from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
    chain,
)

add_1 = RunnableLambda(lambda x: x + 1)
add_3 = RunnableLambda(lambda x: x + 3)


@chain
def mult_2(x: int):
    return x * 2


sequence = mult_2 | add_1
parallel = mult_2 | {"add_1": add_1, "add_3": add_3}

# same as above:
parallel1 = RunnableParallel(add_1=add_1, add_3=add_3)

Run the runnable directly, batched, in parallel (multi-threaded whenever possible!), ...

In [4]:
sequence.invoke(1)  # 3
sequence.batch([1, 2, 3])  # [3,5,7]
parallel.invoke(1)  # {'add_1': 3, 'add_2': 4}

await sequence.abatch([1, 2, 3, 4, 5])

[3, 5, 7, 9, 11]

Runnable can stream their outcome

In [17]:
for s in sequence.stream(100000):
    print(s, end="", flush=True)

200001

Print the graph and various type information

In [7]:
parallel.get_graph().print_ascii()

print("input type:", sequence.InputType)
print("output type:", sequence.OutputType)

print("input schema: ", sequence.input_schema().schema())
print("output schema: ", sequence.output_schema().schema())

                           +--------------+                            
                           | mult_2_input |                            
                           +--------------+                            
                                   *                                   
                                   *                                   
                                   *                                   
                          +----------------+                           
                          | Lambda(mult_2) |                           
                          +----------------+                           
                                   *                                   
                                   *                                   
                                   *                                   
                    +----------------------------+                     
                    | Parallel<add_1,add_3>Input |              

Use RunnablePassthrough

In [8]:
runnable = RunnableParallel(origin=RunnablePassthrough(), modified=add_1)

runnable.invoke(1)  # {'origin': 1, 'modified': 2}

{'origin': 1, 'modified': 2}

Demo 'bind' and 'RunnableConfig' : implement a filter, and log activities

In [9]:
from langchain_core.runnables import RunnableConfig
from loguru import logger


@chain # type: ignore
def max(x: int, max: int, config: RunnableConfig) -> int:
    if log := (config["logger"]):  # type: ignore
        log.info(f"check if {x} < {max}")
    return max if x >= max else x


a = sequence | max.bind(max=6)
a.batch([1, 2, 3, 4, 5], {"logger": logger})

2024-05-05 14:12:40.458 | INFO     | __main__:max:8 - check if 3 < 6
2024-05-05 14:12:40.462 | INFO     | __main__:max:8 - check if 5 < 6
2024-05-05 14:12:40.467 | INFO     | __main__:max:8 - check if 7 < 6
2024-05-05 14:12:40.470 | INFO     | __main__:max:8 - check if 9 < 6
2024-05-05 14:12:40.473 | INFO     | __main__:max:8 - check if 11 < 6


[3, 5, 6, 6, 6]

Demo 'assign", that adds new fields to the dict output of the runnable and returns a new runnable.

In [10]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = (
    RunnableParallel(
        extra=RunnablePassthrough.assign(mult_10=lambda x: x["num"] * 10),
        plus_1=lambda x: x["num"] + 1,
    )
    .assign(info=lambda x: x)
    .assign(plus_1_time_3=lambda x: x["plus_1"] * 3)
)

runnable.invoke({"num": 2})

{'extra': {'num': 2, 'mult_10': 20},
 'plus_1': 3,
 'info': {'extra': {'num': 2, 'mult_10': 20}, 'plus_1': 3},
 'plus_1_time_3': 9}

Most of the time, there are several parameters, or it's a dictionary (almost equivalent in Python).
'itemgetter' create a function that can extract one or several fields from a dictionary. 

First have a look at how it works:

In [11]:
from operator import itemgetter

dic = {
    "question": "What are the types of agent memory?",
    "generation": "The types of agent memory are: sensory memory, short-term memory, and long-term memory.",
    "documents": [],
}

getter_function = itemgetter("generation")
getter_function(dic)

'The types of agent memory are: sensory memory, short-term memory, and long-term memory.'

here an example with Runnables

In [12]:
adder = RunnableLambda(lambda d: d["op1"] + d["op2"])


mult_2_and_add = (
    RunnableParallel(
        op1=RunnablePassthrough() | itemgetter("a") | mult_2,
        op2=RunnablePassthrough() | itemgetter("b") | mult_2,
    )
    | adder
)

mult_2_and_add.invoke({"a": 10, "b": 2})  # should return 2*10 + 2*2 = 24

24

In [16]:
zzz = RunnableLambda(lambda x: {"a": x, "b": 2}) | mult_2_and_add

zzz.invoke(10)

24

Runnables can have fallback in case they break.

In [15]:
@chain
def mult_10_fail(x: int):
    raise Exception("unavailable multiplication by 10 service")
    return x * 10


fallback_chain = mult_10_fail.with_fallbacks([mult_2])
fallback_chain.invoke(2)

4